In [1]:
 current_dir = "/content/gdrive/My Drive/Colab Notebooks/Thesis"
 target_dir = "/content/gdrive/My Drive/Colab Notebooks/Thesis/Politicians"

In [ ]:
import os
from google.colab import drive
import pandas as pd
import re
drive.mount('/content/gdrive')
os.chdir(current_dir)

In [ ]:
ls

In [ ]:
!pip install fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

In [4]:
# Change the current working directory to the 'Politicians' folder
folder_path = target_dir  # Replace with your actual folder path
os.chdir(folder_path)

# List all txt files in the current directory
file_names = [f for f in os.listdir() if f.endswith('.txt')]
file_contents = []

# Read the contents of each txt file
for file_name in file_names:
    with open(file_name, 'r') as file:
        file_contents.append(file.read())

In [5]:
database = {}

for name, content in zip(file_names, file_contents):
    name_without_extension = name.split('.txt')[0]
    database[name_without_extension] = content

In [6]:
database.keys()

dict_keys(['jfk', 'reagan', 'fdr', 'clinton', 'gwb', 'eisenhower', 'lbj', 'ford', 'carter', 'hwb', 'truman', 'hoover', 'cruz', 'sanders', 'trump', 'nixon'])

In [7]:
real_name = {
    'jfk': 'John F. Kennedy',
    'reagan': 'Ronald Reagan',
    'fdr': 'Franklin D. Roosevelt',
    'clinton': 'William Jefferson Clinton',
    'gwb': 'George W. Bush',
    'eisenhower': 'Dwight D. Eisenhower',
    'lbj': 'Lyndon B. Johnson',
    'ford': 'Gerald Ford',
    'carter': 'James Earl Carter',
    'nixon': 'Richard Nixon',
    'hwb': 'George H. Bush',
    'truman': 'Harry S. Truman',
    'hoover': 'Herbert Hoover',
    'cruz': 'Ted Cruz',
    'sanders': 'Bernie Sanders',
    'trump': 'Donald Trump'
}


In [8]:
def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Replace newlines and double whitespaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [9]:
texts = {real_name[k]: clean_text(v) for k, v in database.items()}


In [10]:
for k,v in texts.items():
  print(k, len(v.split()))

John F. Kennedy 809
Ronald Reagan 749
Franklin D. Roosevelt 283
William Jefferson Clinton 685
George W. Bush 399
Dwight D. Eisenhower 576
Lyndon B. Johnson 586
Gerald Ford 507
James Earl Carter 411
George H. Bush 1025
Harry S. Truman 970
Herbert Hoover 1500
Ted Cruz 812
Bernie Sanders 832
Donald Trump 1761
Richard Nixon 1551


In [11]:
dutton_scores = {
    'Donald Trump': 22.67,
    'Ted Cruz': 20.33,
    'Bernie Sanders': 18.0
}


In [12]:
lilienfeld_scores = {
    "Theodore Roosevelt": 1.462,
    "John F. Kennedy": 1.408,
    "Franklin D. Roosevelt": 1.079,
    "Ronald Reagan": 0.912,
    "Rutherford B. Hayes": 0.824,
    "Zachary Taylor": 0.671,
    "William Jefferson Clinton": 0.569,
    "Martin Van Buren": 0.554,
    "Andrew Jackson": 0.516,
    "George W. Bush": 0.391,
    "George Washington": 0.302,
    "Dwight D. Eisenhower": 0.297,
    "John Tyler": 0.283,
    "Chester Arthur": 0.267,
    "Lyndon B. Johnson": 0.173,
    "Gerald Ford": 0.157,
    "Benjamin Harrison": 0.032,
    "James Earl Carter": 0.007,
    "Woodrow Wilson": -0.032,
    "Warren G. Harding": -0.036,
    "Thomas Jefferson": -0.056,
    "Ulysses S. Grant": -0.084,
    "William H. Harrison": -0.158,
    "Abraham Lincoln": -0.321,
    "James Madison": -0.355,
    "Millard Fillmore": -0.388,
    "James K. Polk": -0.388,
    "Richard Nixon": -0.544,
    "Franklin Pierce": -0.553,
    "George H. Bush": -0.619,
    "Grover Cleveland": -0.624,
    "James Monroe": -0.636,
    "James Garfield": -0.664,
    "Harry S. Truman": -0.668,
    "Andrew Johnson": -0.728,
    "Herbert Hoover": -0.866,
    "John Adams": -0.927,
    "James Buchanan": -0.942,
    "William McKinley": -0.996,
    "Calvin Coolidge": -1.175,
    "John Q. Adams": -1.234,
    "William H. Taft": -1.579
}


#Lilienfeld Model

In [13]:
lil_dict = {}
lil_dict["presidents"] = [name for name,score in lilienfeld_scores.items() if name in texts.keys()]
lil_dict["scores"] = [v for k,v in lilienfeld_scores.items() if k in texts.keys()]
lil_dict["texts"] = [texts[k] for k,v in lilienfeld_scores.items() if k in texts.keys()]


In [14]:
lil_df = pd.DataFrame(lil_dict)

In [16]:
lil_df

presidents  scores  \
0             John F. Kennedy   1.408   
1       Franklin D. Roosevelt   1.079   
2               Ronald Reagan   0.912   
3   William Jefferson Clinton   0.569   
4              George W. Bush   0.391   
5        Dwight D. Eisenhower   0.297   
6           Lyndon B. Johnson   0.173   
7                 Gerald Ford   0.157   
8           James Earl Carter   0.007   
9               Richard Nixon  -0.544   
10             George H. Bush  -0.619   
11            Harry S. Truman  -0.668   
12             Herbert Hoover  -0.866   

                                                texts  
0   In the 1st place I think the problems are more...  
1   Hello Hello Greta I did I called with the fact...  
2   Well She core of course was a major part I had...  
3   Most people think you know that uh except for ...  
4   Well thanks Rudy and thanks George Let me make...  
5   Well I dont know As far as Vice president Nixo...  
6   I want a couple of maybe 3 of the light brown ...  
7   It was August 1 1974 and uh general Hague who ...  
8   Well there are varying degrees of relationship...  
9   Because the United States is concerned by more...  
10  I dont I honestly Larry I dont I had my chance...  
11  Well I I itll sound as if Im Bragging about th...  
12  Republic after the First World War After the F...

In [ ]:
import pandas as pd
import pickle

# Assuming you have a DataFrame called "lil_df"
# Save the DataFrame to a pickle file
with open('lil_df.pickle', 'wb') as f:
    pickle.dump(lil_df, f)
